<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/air-2024/blob/main/notebooks/4_przestrzenne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install libspatialindex-dev ## specjalna biblioteka do analiz przestrzennych na linux
!pip install geopandas ## biblioteka geopandas w pythonie
!pip install folium mapclassify ## potrzebne do utworzenia mapy interaktywnej
## doinstalowujemy nowe pakiety na potrzeby analiz przestrzennych
!pip install -U pysal urbanaccess pandana libpysal esda

In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium ## folium.Map
import libpysal as ps ## wagi
import esda ## statystyka morana
import mapclassify
from splot.esda import lisa_cluster ## wykres: kartogram ze skupiskami
from splot.esda import moran_scatterplot ## wykres: wykres rozrzutu ze statystyką morana
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = [8, 8] ## wielkość wykresu

In [ ]:
pow_df = gpd.read_file(filename = "powiaty.shp", encoding = "utf-8")
pow_df = pow_df[["SP_ID", "jpt_kod_je", "jpt_nazwa_", "geometry"]]
pow_df.head(n = 2)

In [ ]:
pow_df.crs

In [ ]:
pow_df.set_crs(2180, inplace = True)

In [ ]:
pow_df.crs

In [ ]:
pow_df.plot(color = "black", edgecolor = "white")

In [ ]:
pow_df[pow_df.jpt_kod_je.str.slice(0,2) == "30"].plot(color = "white", edgecolor = "black")

In [ ]:
pow_df[pow_df.jpt_kod_je.str.slice(0,2) == "30"].explore()

In [ ]:
pzn_df = gpd.read_file(filename = "Poznan.shp", encoding = "utf-8")
pzn_df.head(n = 2)

In [ ]:
pzn_df.crs

In [ ]:
pzn_df.plot()

In [ ]:
pow_df.crs.srs

In [ ]:
pzn_df.crs.srs

In [ ]:
pzn_df.set_crs(4326, allow_override=True, inplace=True)

In [ ]:
pzn_df.crs

In [ ]:
pzn_df.plot()

In [ ]:
pzn_df.to_crs(2180, inplace = True)

In [ ]:
pzn_df.head()

In [ ]:
pzn_df.explore()

In [ ]:
pzn_df.plot(color = "white", edgecolor = "black")

In [ ]:
base = pow_df[pow_df.jpt_kod_je == "3021"].plot(color = "white", edgecolor = "black")

pzn_df.plot(ax=base, color = "white", edgecolor = "black")

In [ ]:
woj = gpd.read_file("woj.shp", encoding = "utf-8")
woj.set_crs(2180, inplace = True)
pow_base = pow_df.plot(color = "none", edgecolor = "black", linewidth = 0.4)
woj.plot(ax=pow_base, color = "none",  edgecolor = "black", linewidth = 1.2).set_axis_off()

In [ ]:
wynag = pd.read_excel(io = "powiaty_wynag.xlsx", sheet_name=1)
wynag = wynag[["Kod", "Wartosc"]]
wynag["kod"] = wynag.Kod.astype(str).str.zfill(7)
wynag["kod"] = wynag.kod.str.slice(start=0,stop=4)
wynag.head(n=2)

In [ ]:
pow_df_dane = pd.merge(left = pow_df,
                       right = wynag[["kod", "Wartosc"]],
                       how = "left",
                       left_on = "jpt_kod_je",
                       right_on = "kod")
pow_df_dane.head(n=3)

In [ ]:
pow_df_dane.plot(column = "Wartosc", cmap = "magma_r",
                 edgecolor = "black", linewidth = 0.4,
                 legend = True, legend_kwds = {'shrink': 0.5})
#                 legend = True,  legend_kwds = {'shrink': 0.6})